# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
# [TODO] Define a query and run
query = "What's the newest Google Pixel phone?"
result = client.search(query)

In [5]:
result

{'query': "What's the newest Google Pixel phone?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://ghostek.com/blogs/ghostek-insider/whats-the-newest-pixel-phone-and-is-it-worth-buying?srsltid=AfmBOorJXJNhqIW2ZHMoqj5Z3iHM01JdaHJybmH_gNXI64xSHMMLuCby',
   'title': "What's the Newest Pixel Phone and Is It Worth Buying?",
   'content': 'As of November 2025, the latest Google Pixel phones are those in the Pixel 10 series: the base model, Pixel 10 Pro, Pro XL, and the 10 Pro Fold.',
   'score': 0.92359936,
   'raw_content': None},
  {'url': 'https://www.bestbuy.com/site/google/learn-about-google-pixel/pcmcat1566430029496.c?id=pcmcat1566430029496',
   'title': 'Google Pixel: Newest Pixel Phone',
   'content': '# Google Pixel. Upon trade-in of an eligible phone, customer will receive value toward Google Pixel 10 Pro Fold, Pixel 10 Pro XL, Pixel 10 Pro, and Pixel 10 with trade-in. $760 total savings based on trade-in value with Google Pixel 9 Pro XL 

## Define Web Search tool

In [18]:
# [TODO] Define the search tool
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web for the latest information on a given topic
    args:
        query (str): The topic to search for
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)

    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }

    return formatted_results    

In [19]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [20]:
# [TODO] Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful assistant that can answer questions about the world."    
)

In [21]:
# [TODO] Agent with web search tool
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful assistant that can answer questions about the world.",
    tools=[web_search]
)

In [22]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [13]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant that can answer questions about the world., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I cannot provide information on events or awards that occurred after October 2023, including the 2025 Oscars. You may want to check the latest updates from reliable news sources or the official Oscars website for the most current information., tool_calls = None)


In [14]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I cannot provide information on events or awards that occurred after October 2023, including the 2025 Oscars. You may want to check the latest updates from reliable news sources or the official Oscars website for the most current information.


In [15]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant that can answer questions about the world., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I cannot provide information on events or awards that occurred after October 2023, including the 2025 Oscars. You may want to check the latest updates from reliable news sources or the official Oscars website for the most current information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2023, several significant developments in AI technology were making headlines. Some of the notable tren

In [16]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2023, several significant developments in AI technology were making headlines. Some of the notable trends and advancements included:

1. **Generative AI**: Models like GPT-4 and advancements in image generation (e.g., DALL-E) continued to evolve, enabling more sophisticated content creation across various media.

2. **AI Ethics and Regulation**: There was an increasing focus on the ethical implications of AI, with discussions around regulation and responsible AI usage gaining traction. Many governments and organizations were outlining frameworks to ensure AI is developed and used ethically.

3. **AI in Healthcare**: AI applications in healthcare were expanding, with improved diagnostic tools, personalized medicine, and predictive analytics helping in patient care and operational efficiency.

4. **Natural Language Processing (NLP)**: Advances in NLP were improving the ability of AI to understand and generate human language, enhancing application

**Web Agent**

In [23]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant that can answer questions about the world., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_8vOEVRdDix1b5FoB6p3BUX37', function=Function(arguments='{"query":"2025 Oscar for International Movie winner","search_depth":"advanced"}', name='web_search'), type='function')])
 -> (role = tool, content = "{'answer': 'The 2025 Oscar for Best International Feature went to Brazil\\'s \"I\\'m Still Here.\" The film received three nominations, including Best Picture and Best Actress for Fernanda Torres. 

In [24]:
print(run1.get_final_state()["messages"][-1].content)

The 2025 Oscar for Best International Feature went to Brazil's film "I'm Still Here." The film received three nominations, including Best Picture and Best Actress for Fernanda Torres. This win was celebrated for its powerful message. 

For more details, you can check out the article on [NPR](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature) or [People](https://people.com/oscars-2025-im-still-here-brazil-best-international-feature-11684447).


In [25]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant that can answer questions about the world., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageFunctionToolCall(id='call_8vOEVRdDix1b5FoB6p3BUX37', function=Function(arguments='{"query":"2025 Oscar for International Movie winner","search_depth":"advanced"}', name='web_search'), type='function')])
 -> (role = tool, content = "{'answer': 'The 2025 Oscar for Best International Feature went to Brazil\\'s \"I\\'m Still Here.\" The film received three nominations, including Best Picture and Best Actress for Fernanda Torres. 

In [26]:
print(run2.get_final_state()["messages"][-1].content)

As of 2025, several significant developments in AI technology have taken place:

1. **Proactive AI Systems**: AI is evolving from reactive to proactive systems. New deep learning models are showing substantial improvements in reasoning tasks across various domains, including math and science, enabling real-time analysis and more accurate results.

2. **Embedded Intelligence**: AI is increasingly becoming a part of everyday life, impacting industries like healthcare and transportation. By 2023, the FDA had approved numerous AI-enabled medical devices, and autonomous vehicles have become operational in various cities.

3. **Major Company Innovations**: Companies like Google have made numerous announcements related to AI, including updates to their Gemini product and new AI features in search functionalities. There is a strong focus on creating smarter systems that enhance decision-making.

4. **AI in Automation**: The automation of routine tasks has been a significant trend, with AI bein

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow